# Next TODO: 

I need a working deck, that I can learn with <b>as fast as possible!</b>

- <span style="color:lightgreen">Anki card styling!</span> Like 'the NewYorker' or 'Economist'! Black fonts, different sizes, like a newspaper! With thin lines between definition, hint, notes...
- add <span style="color:lightgreen">『』, "" </span> to long_sentence for better styling
- load <span style="color:lightgreen">"native_langage"</span> as a column and have a native_message for everything!  
- <span style="color:lightgreen">ふりがな</span> with a python library, not in Anki. Apparently 'Sudachi' is very reliable... Mecab. (kodachi) Have <span style="color:lightgreen">furigana</span> in separate code!
- maybe include a japanese dictionary like https://pypi.org/project/jamdict/ to improve the definitions. Which dictionaries should I rely on? API, downloaded html dictionary or something else? What does Lisardo say?

## Wish list: 
- more synonyms, always also in target_language
- related phrases, common phrases....
- create a gpt assistant that gives better definitions
- more Kanji information
Different information depending on word_type:
    + nouns: related words, びょういん and 病気　etc,
    + verbs: conjugation, て-Form! (to know if る or う), prepositions for the verbs...
    + adjectives: な or い, conjugation examples


In [1]:
import json
import genanki
import re
import pandas as pd
from dotenv import load_dotenv
import os
import openai
from jinja2 import Template

json_file_path = '/Users/moritzvitt/coding/jn_config.json'
csv_file_path = '/Users/moritzvitt/downloads/anki/items.csv'

# Open the JSON file and load its contents into a Python dictionary
with open(json_file_path, 'r') as json_file:
    config_json = json.load(json_file)

# chatGpt configuration
load_dotenv()
openai.api_key = os.getenv("22API_KEY")

load_dotenv()
ai_model = os.getenv("model")

# Define the column names of your CSV file
column_names = config_json['column_names']
df = pd.read_csv(csv_file_path, delimiter='\t', names=column_names)

# drop all rows where word_or_phrase is phrase
df = df[df['word_or_phrase'] != 'Phrase']

# native_language column

df['native_language'] = config_json['native_language']
# print(df.columns)
df.head(2)

/Users/moritzvitt/coding/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


,ID,word_or_phrase,short_phrase,short_translation,word,stem,word_type,NAN,word_translation,website,...,movie_code,source,date,long_phrase,machine_translation,human_translation,first_jpg,second_jpg,audio,native_language
0,WORD|守る|ja,Word,その代わり その子を しっかり守るんだよ,"Instead, we're going to protect that child.",守る,守る,Verb,NaN,"protect, defend, guard",Netflix,...,60023642,Spirited Away,2024-01-26 13:02,ハク龍(りゅう) あなたのした事は もうとがめません\nその代わり その子を しっかり守るん...,"I won't do what you've done.\nInstead, we're g...","Haku, I no longer blame you for what you did.\...",1706274142813_prev.jpg,1706274142813_next.jpg,1706274142813.mp3,english
1,WORD|子|ja,Word,その代わり その子を しっかり守るんだよ,"Instead, we're going to protect that child.",子,子,Noun,NaN,"child, kid, son",Netflix,...,60023642,Spirited Away,2024-01-26 13:02,ハク龍(りゅう) あなたのした事は もうとがめません\nその代わり その子を しっかり守るん...,"I won't do what you've done.\nInstead, we're g...","Haku, I no longer blame you for what you did.\...",1706274141384_prev.jpg,1706274141384_next.jpg,1706274141384.mp3,english


# creating the 'messages', to send to chatGPT, with a templating library

In [2]:
df_messages = pd.DataFrame()

def load_and_resolve_template(template_dict, template_string):
    template = Template(template_string)
    resolved_template = template.render(template_dict)
    return resolved_template

for message in config_json["messages"]:
    message_name = f"{message}"
    df_messages[message_name] = df.apply(lambda row: load_and_resolve_template(row.to_dict(), config_json["messages"][message_name]), axis=1)

def filter_fields(df_messages, wanted_fields):
    for field, value in wanted_fields.items():
        if not value:
            print(field)
            df_messages.drop(field, axis=1, inplace=True)
            
        else:
            pass
filter_fields(df_messages, config_json["wanted_fields"])

df_messages.head()
print(df_messages)

pd.set_option('display.max_columns', None)  # None means unlimited
pd.set_option('display.expand_frame_repr', False)  # Don't wrap to multiple lines
pd.set_option('display.max_colwidth', None)  # None means unlimited

print(df_messages)

grammar
native_synonyms
native_explanation
native_examples
native_hint
native_definition
native_grammar
                                            synonyms  \
0  Provide me with 2 synonyms for 守る in english. ...   
1  Provide me with 2 synonyms for 子 in english. Y...   
2  Provide me with 2 synonyms for 代わり in english....   
3  Provide me with 2 synonyms for 事 in english. Y...   
4  Provide me with 2 synonyms for 生き in english. ...   
5  Provide me with 2 synonyms for タイミング in englis...   

                                            examples  \
0  Give me 2 more examples in japanese for the us...   
1  Give me 2 more examples in japanese for the us...   
2  Give me 2 more examples in japanese for the us...   
3  Give me 2 more examples in japanese for the us...   
4  Give me 2 more examples in japanese for the us...   
5  Give me 2 more examples in japanese for the us...   

                                         explanation  \
0  Please answer in english. Provide me with a br...  

# Creating the 'content' with chatGPT

In [3]:
df_anki = pd.DataFrame()
def generate_content(message):
    response = openai.ChatCompletion.create(
        model=ai_model,
        messages=[{"role": "user", "content": message}],
        max_tokens=50,
        # temperature=0.5
    )
    content = response.choices[0].message["content"]
    print(content)
    return content

for column in df_messages.columns:
    df_anki[column] = df_messages.apply(lambda row: generate_content(row[column]), axis=1)

df_anki.head

df_anki = pd.concat([df_anki, df[['ID', 'word', 'audio', 'first_jpg', 'second_jpg', 'long_phrase', 'human_translation', 'word_translation', 'source', 'date']]], axis=1)
df_anki.head(2)

protect, defend
child, kid
replacement, substitute
thing, deed
alive, living
timing, moment
1. 法律を守るべきだ。
   ほうりつをまもるべきだ。

2. 自然環境を守ることは大切
1. 子犬 (こいぬ)
2. 子供 (こども)
1. 今日は体調が悪いので、代わりに弟が買い物に行ってくれました。
   きょうはたいちょうがわ
1. 彼女は料理が上手なことで知られています。
   (かのじょはりょうりがじょうずなことでしられています
1. 生きがいを見つけるために旅に出た。
   (いきがいをみつけるためにたびにでた。)

2. 生き抜く
1. タイミングが合(あ)わなくて、なかなか友達(ともだち)に会(あ)えない。
2. ボールを受け取
守る: means "to protect" or "to defend" someone or something.
子: in context, refers to 'child/kid/person'; "protect that child/person well" implied.
代わり: Japanese for "instead" or "in exchange"; implies a condition or substitution.
事: In this context, means "deeds" or "actions".
生き: Japanese verb root meaning "live" or "exist," as in "Grandma Haku was alive."
タイミング: Japanese term for 'timing'; in 'グッドタイミングね' means 'good timing' or 'well-timed moment'.
守る: protect or guard the child firmly.
子: child or that person (contextual).
代わり: in exchange, on the condition.
事: deed/action (context: no longer blame).
生き: 'to live', 'livi

,synonyms,examples,explanation,hint,definition,ID,word,audio,first_jpg,second_jpg,long_phrase,human_translation,word_translation,source,date
0,"protect, defend",1. 法律を守るべきだ。\n ほうりつをまもるべきだ。\n\n2. 自然環境を守ることは大切,"守る: means ""to protect"" or ""to defend"" someone or something.",守る: protect or guard the child firmly.,守る: to protect or defend someone or something.,WORD|守る|ja,守る,1706274142813.mp3,1706274142813_prev.jpg,1706274142813_next.jpg,ハク龍(りゅう) あなたのした事は もうとがめません\nその代わり その子を しっかり守るんだよ\nさあ 坊(ぼう)やたち お帰りの時間だよ,"Haku, I no longer blame you for what you did.\nJust be sure you protect the girl.\nOK, boys, time to go home.","protect, defend, guard",Spirited Away,2024-01-26 13:02
1,"child, kid",1. 子犬 (こいぬ)\n2. 子供 (こども),"子: in context, refers to 'child/kid/person'; ""protect that child/person well"" implied.",子: child or that person (contextual).,"子: In this context, means 'child/kid'; generally, it signifies 'child', 'son' or 'suffix for males'.",WORD|子|ja,子,1706274141384.mp3,1706274141384_prev.jpg,1706274141384_next.jpg,ハク龍(りゅう) あなたのした事は もうとがめません\nその代わり その子を しっかり守るんだよ\nさあ 坊(ぼう)やたち お帰りの時間だよ,"Haku, I no longer blame you for what you did.\nJust be sure you protect the girl.\nOK, boys, time to go home.","child, kid, son",Spirited Away,2024-01-26 13:02


# Formatting hint, definition and notes fields!

In [4]:
def add_fields(df_anki, wanted_fields):
    for field, should_add in wanted_fields.items():
        if not should_add:
            field_name = f"{field}"
            df_anki[field_name] = ""

add_fields(df_anki, config_json["wanted_fields"])

import random

def createID(ID, source, date):
    newID = f"{ID}{source}{date}"
    newID = newID.replace(" ", "__")
    #random number
    newID = newID + "__" + str(random.randint(1000, 9999))
    return newID
df_anki["ID"] = df_anki.apply(lambda row: createID(row["ID"], row["source"], row["date"]), axis=1)

def mp3_pattern(mp3):
        mp3 = "[sound:" + mp3 + "]"
        return mp3
df_anki["audio"] = df_anki.apply(lambda row: mp3_pattern(row["audio"]), axis=1)

def image(first_jpg, second_jpg):
    image = f'<img src="{first_jpg}"><img src="{second_jpg}">'
    return image
df_anki["image"] = df_anki.apply(lambda row: image(row["first_jpg"], row["second_jpg"]), axis=1)

def clozePattern(word, phrase, synonyms):
    word_new = '{' + '{' f'c1::{word}::{synonyms}' + '}' + '}'
    print(word_new)

    phrase = re.sub(f'{word}', word_new, phrase, flags=re.IGNORECASE)
    print(phrase)
    return phrase

df_anki["cloze"] = df_anki.apply(lambda row: clozePattern(row["word"], row["long_phrase"], row["synonyms"]), axis=1)

def cleanHint(hint):
    # print(type(hint))
    if hint != "" and ":" in hint:
        parts = re.findall(r"[:](.+)?", hint) 
        # r"[:](.+)?" worked
        return f": {parts[0]}"
    else: 
        return ""
df_anki["hint"] = df_anki["hint"].apply(cleanHint)

def definition_field(definition, explanation, grammar):
    print("definition")
    print(f"{str(definition)}")
    newDefinition = str(definition) + "<br>" + str(explanation) + "<br>" + str(grammar) + "<br>"
    return newDefinition
df_anki["definition"] = df_anki.apply(lambda row: definition_field(row["definition"], row["explanation"], row["grammar"]), axis=1)

def notes_field(translation, word_translation):
    translation_str = str(translation) if not pd.isnull(translation) else ""
    word_translation_str = str(word_translation) if not pd.isnull(word_translation) else ""
    newNotes = "sentence_meaning:" + translation_str + "<br>word_meaning:" + word_translation_str + "<br>"
    return newNotes

df_anki["notes"] = df_anki.apply(lambda row: notes_field(row["human_translation"], row["word_translation"]), axis=1)

df_anki[["ID", "cloze", "hint", "definition", "image", "audio"]].head(2)

{{c1::守る::protect, defend}}
ハク龍(りゅう) あなたのした事は もうとがめません
その代わり その子を しっかり{{c1::守る::protect, defend}}んだよ
さあ 坊(ぼう)やたち お帰りの時間だよ
{{c1::子::child, kid}}
ハク龍(りゅう) あなたのした事は もうとがめません
その代わり その{{c1::子::child, kid}}を しっかり守るんだよ
さあ 坊(ぼう)やたち お帰りの時間だよ
{{c1::代わり::replacement, substitute}}
ハク龍(りゅう) あなたのした事は もうとがめません
その{{c1::代わり::replacement, substitute}} その子を しっかり守るんだよ
さあ 坊(ぼう)やたち お帰りの時間だよ
{{c1::事::thing, deed}}
おばあちゃん ハク生きてた
ハク龍(りゅう) あなたのした{{c1::事::thing, deed}}は もうとがめません
その代わり その子を しっかり守るんだよ
{{c1::生き::alive, living}}
グッドタイミングね
おばあちゃん ハク{{c1::生き::alive, living}}てた
ハク龍(りゅう) あなたのした事は もうとがめません
{{c1::タイミング::timing, moment}}
よかった
グッド{{c1::タイミング::timing, moment}}ね
おばあちゃん ハク生きてた
definition
守る: to protect or defend someone or something.
definition
子: In this context, means 'child/kid'; generally, it signifies 'child', 'son' or 'suffix for males'.
definition
代わり: "in exchange for; instead" (as the compensation for protecting the child).
definition
事: in this context, it means 'what you have done' or 'deed'.
defini

,ID,cloze,hint,definition,image,audio
0,WORD|守る|jaSpirited__Away2024-01-26__13:02__2997,"ハク龍(りゅう) あなたのした事は もうとがめません\nその代わり その子を しっかり{{c1::守る::protect, defend}}んだよ\nさあ 坊(ぼう)やたち お帰りの時間だよ",: protect or guard the child firmly.,"守る: to protect or defend someone or something.<br>守る: means ""to protect"" or ""to defend"" someone or something.<br><br>","<img src=""1706274142813_prev.jpg""><img src=""1706274142813_next.jpg"">",[sound:1706274142813.mp3]
1,WORD|子|jaSpirited__Away2024-01-26__13:02__7203,"ハク龍(りゅう) あなたのした事は もうとがめません\nその代わり その{{c1::子::child, kid}}を しっかり守るんだよ\nさあ 坊(ぼう)やたち お帰りの時間だよ",: child or that person (contextual).,"子: In this context, means 'child/kid'; generally, it signifies 'child', 'son' or 'suffix for males'.<br>子: in context, refers to 'child/kid/person'; ""protect that child/person well"" implied.<br><br>","<img src=""1706274141384_prev.jpg""><img src=""1706274141384_next.jpg"">",[sound:1706274141384.mp3]


In [1]:
# Read content from the external file
with open('anki.html', 'r', encoding='utf-8') as content_file:
    content = content_file.read()

# Splitting HTML content
html_sections = content.split('<!-- html -->')

# Assigning sections to qfmt, afmt, and css
qfmt_html = html_sections[1]
afmt_html = html_sections[2]
css_code = html_sections[3]

df = df_anki[["ID", "cloze", "hint", "definition", "notes", "image", "audio"]]
# so no problem later with the anki package, to string
df = df.astype(str)

# Define the Anki model
model_id = 1607392319
model = genanki.Model(
    model_id,
    'Cloze Deletion Model',
    fields=[
        {'name': 'ID'},
        {'name': 'cloze'},
        {'name': 'hint'},
        {'name': 'definition'},
        {'name': 'notes'},
        {'name': 'image'},
        {'name': 'audio'},
    ],

    templates=[
        {
            'name': 'Card 1',
            'qfmt': qfmt_html,
            'afmt': afmt_html,
        },
    ],
    css=css_code
)

# Create an Anki deck
deck_id = model_id + 1  # Ensure deck_id is different from model_id
deck = genanki.Deck(deck_id, 'lln_anki_deck')

# Add cards to the deck
# Add cards to the deck
for index, row in df.iterrows():
    my_note = genanki.Note(
        model=model,
        fields=[(row['ID']), (row['cloze']), (row['hint']), (row['definition']), (row['notes']), (row['image']), (row['audio'])],
    )
    deck.add_note(my_note)


# Specify the desktop path
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")

# Create Anki package on the desktop
package_path = os.path.join(desktop_path, 'lln_anki_deck.apkg')
package = genanki.Package(deck)
package.write_to_file(package_path)
print(f'Anki package "{package_path}" has been created.')


NameError: name 'df_anki' is not defined